# Procesamiento de imágenes a través de morfología matemática 

<div class="alert alert-block alert-success">
<b>Resumen:</b> El código implementa operaciones de morfología matemática en imágenes binarias utilizando OpenCV, NumPy y Matplotlib en Python; crea elementos estructurantes personalizados (diamante, disco, octágono, línea) y utiliza predefinidos (rectángulo, cuadrado) para aplicar operaciones morfológicas básicas como dilatación y erosión, así como operaciones compuestas como apertura y cierre, demostrando sus efectos en diversas imágenes y aplicándolas en casos prácticos.
</div>

***

## Índice de contenidos

Preámbulo
1. Elementos estructurantes 2D.
2. Operación de dilatación.
    - Aplicación en análisis de documentos.
4. Erosión.
5. Opening.
6. Closing.
7. Opening y closing.
8. Aplicaciones.
    - Mejora de una huella digital ruidosa.

---

# Preámbulo

In [ ]:
# Limpieza del entorno y carga de librerías
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 1. Elementos estructurantes 2D

En esta sección, crearemos diversos elementos estructurantes que se utilizan en operaciones morfológicas.

In [ ]:
# Funciones para crear elementos estructurantes personalizados

def diamond(r):
    L = np.arange(-r, r+1)
    X, Y = np.meshgrid(L, L)
    mask = (np.abs(X) + np.abs(Y)) <= r
    return mask.astype(np.uint8)

def disk(r):
    L = np.arange(-r, r+1)
    X, Y = np.meshgrid(L, L)
    mask = X**2 + Y**2 <= r**2
    return mask.astype(np.uint8)

def octagon(r):
    size = 2*r + 1
    mask = np.ones((size, size), dtype=np.uint8)
    for i in range(size):
        for j in range(size):
            if (i < r - r//2 and j < r - r//2) or \
               (i < r - r//2 and j > r + r//2) or \
               (i > r + r//2 and j < r - r//2) or \
               (i > r + r//2 and j > r + r//2):
                mask[i, j] = 0
    return mask

def line_element(length, angle):
    angle_rad = np.deg2rad(angle)
    x = int(length * np.cos(angle_rad))
    y = int(length * np.sin(angle_rad))
    size = max(abs(x), abs(y)) * 2 + 1
    element = np.zeros((size, size), dtype=np.uint8)
    cv2.line(element, (size//2 - x, size//2 - y), (size//2 + x, size//2 + y), 1, thickness=1)
    return element

Parámetros para los elementos estructurantes

In [ ]:
r = 9
m = 7
n = 4
w = 7
length = 7
angle = 45

Creación de los elementos estructurantes

In [ ]:
SE_1 = diamond(r)
SE_2 = disk(r)
SE_3 = octagon(r)
SE_4 = line_element(length, angle)
SE_5 = cv2.getStructuringElement(cv2.MORPH_RECT, (m, n))
SE_6 = cv2.getStructuringElement(cv2.MORPH_RECT, (w, w))

Visualización de los elementos estructurantes

In [ ]:
plt.figure(figsize=(8, 8))

plt.subplot(3, 2, 1)
plt.imshow(SE_1, cmap='gray')
plt.title('Diamante')

plt.subplot(3, 2, 2)
plt.imshow(SE_2, cmap='gray')
plt.title('Disco')

plt.subplot(3, 2, 3)
plt.imshow(SE_3, cmap='gray')
plt.title('Octágono')

plt.subplot(3, 2, 4)
plt.imshow(SE_4, cmap='gray')
plt.title('Línea')

plt.subplot(3, 2, 5)
plt.imshow(SE_5, cmap='gray')
plt.title('Rectángulo')

plt.subplot(3, 2, 6)
plt.imshow(SE_6, cmap='gray')
plt.title('Cuadrado')

plt.tight_layout()
plt.show()

## 2. Operación de dilatación
### 2.1 Ejemplo

Cargamos una imagen binaria y aplicamos una dilatación con un elemento estructurante.

In [ ]:
# Carga de la imagen binaria
BW1 = cv2.imread('./images/U_bw.tif', cv2.IMREAD_GRAYSCALE)

# Asegurarse de que la imagen es binaria
_, BW1 = cv2.threshold(BW1, 127, 255, cv2.THRESH_BINARY)

In [ ]:
# Aplicación de la dilatación
BW1_dil_1 = cv2.dilate(BW1, SE_1, iterations=1)

In [ ]:
# Visualización de los resultados
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(BW1, cmap='gray')
plt.title('Imagen Original')

plt.subplot(1, 2, 2)
plt.imshow(BW1_dil_1, cmap='gray')
plt.title('Imagen Dilatada')

plt.tight_layout()
plt.show()

### 2.2 Aplicación en análisis de documentos

En este ejemplo, utilizamos un elemento estructurante en forma de cruz para dilatar una imagen de texto.

In [ ]:
# Carga de la imagen binaria
BW2 = cv2.imread('./images/text_image.tif', cv2.IMREAD_GRAYSCALE)

# Asegurarse de que la imagen es binaria
_, BW2 = cv2.threshold(BW2, 127, 255, cv2.THRESH_BINARY)

In [ ]:
# Creación del elemento estructurante en forma de cruz
se_plus = np.array([[0, 1, 0],
                 [1, 1, 1],
                 [0, 1, 0]], dtype=np.uint8)

In [ ]:
# Aplicación de la dilatación
BW2_dil_1 = cv2.dilate(BW2, se_plus, iterations=1)

In [ ]:
# Visualización de los resultados
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(BW2, cmap='gray')
plt.title('Imagen Original')

plt.subplot(1, 2, 2)
plt.imshow(BW2_dil_1, cmap='gray')
plt.title('Imagen Dilatada')

plt.tight_layout()
plt.show()

## 3. Erosión
### 3.1 Ejemplo

En esta sección, aplicaremos la operación de erosión a una imagen binaria utilizando elementos estructurantes en forma de disco de diferentes tamaños.

In [ ]:
# Carga de la imagen binaria
BW3 = cv2.imread('./images/wirebond-mask.tif', cv2.IMREAD_GRAYSCALE)
#BW3 = cv2.imread('circles.png', cv2.IMREAD_GRAYSCALE)

# Asegurarse de que la imagen es binaria
_, BW3 = cv2.threshold(BW3, 127, 255, cv2.THRESH_BINARY)

In [ ]:
se_3 = disk(5)
se_4 = disk(7)
se_5 = disk(19)

In [ ]:
# Aplicación de la erosión con los diferentes elementos estructurantes
BW3_ero_1 = cv2.erode(BW3, se_3, iterations=1)
BW3_ero_2 = cv2.erode(BW3, se_4, iterations=1)
BW3_ero_3 = cv2.erode(BW3, se_5, iterations=1)

In [ ]:
# Visualización de los resultados
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.imshow(BW3, cmap='gray')
plt.title('Imagen Original')

plt.subplot(2, 2, 2)
plt.imshow(BW3_ero_1, cmap='gray')
plt.title('Erosión con disco de radio 5')

plt.subplot(2, 2, 3)
plt.imshow(BW3_ero_2, cmap='gray')
plt.title('Erosión con disco de radio 7')

plt.subplot(2, 2, 4)
plt.imshow(BW3_ero_3, cmap='gray')
plt.title('Erosión con disco de radio 19')

plt.tight_layout()
plt.show()

## 4. Opening
### 4.1 Ejemplo

En esta sección, aplicaremos la operación morfológica de apertura (opening) a una imagen binaria utilizando un elemento estructurante en forma de cuadrado. La apertura es útil para eliminar pequeños objetos o detalles en la imagen mientras se preserva la forma y tamaño de los objetos más grandes.

In [ ]:
# Carga de la imagen binaria
BW4 = cv2.imread('./images/shapes.tif', cv2.IMREAD_GRAYSCALE)

# Asegurarse de que la imagen es binaria
_, BW4 = cv2.threshold(BW4, 127, 255, cv2.THRESH_BINARY)

In [ ]:
# Creación del elemento estructurante en forma de cuadrado de tamaño 20x20
se_6 = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 20))

In [ ]:
# Aplicación de la operación de apertura (erosión seguida de dilatación)
BW4_opening_1 = cv2.morphologyEx(BW4, cv2.MORPH_OPEN, se_6)

In [ ]:
# Visualización de los resultados
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(BW4, cmap='gray')
plt.title('Imagen Original')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(BW4_opening_1, cmap='gray')
plt.title('Imagen después de Opening')
plt.axis('off')

plt.tight_layout()
plt.show()

## 5. Closing
### 5.1 Ejemplo

En esta sección, aplicaremos la operación morfológica de cierre (closing) a la misma imagen binaria utilizando el mismo elemento estructurante en forma de cuadrado. El cierre es útil para rellenar pequeños agujeros en objetos y para conectar regiones adyacentes.

In [ ]:
# Aplicación de la operación de cierre (dilatación seguida de erosión)
BW4_closing_1 = cv2.morphologyEx(BW4, cv2.MORPH_CLOSE, se_6)

In [ ]:
# Visualización de los resultados
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(BW4, cmap='gray')
plt.title('Imagen Original')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(BW4_closing_1, cmap='gray')
plt.title('Imagen después de Closing')
plt.axis('off')

plt.tight_layout()
plt.show()

## 6. Opening y closing

En esta sección, aplicaremos una secuencia de operaciones morfológicas a la imagen: primero una apertura y luego un operación cierre, utilizando el mismo elemento estructurante en forma de cuadrado. Esta combinación es útil para eliminar pequeños objetos y suavizar contornos irregulares en una imagen binaria.

In [ ]:
# Aplicación de la operación de cierre sobre la imagen ya abierta
BW4_closing_2 = cv2.morphologyEx(BW4_opening_1, cv2.MORPH_CLOSE, se_6)

In [ ]:
# Visualización de los resultados
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(BW4, cmap='gray')
plt.title('Imagen Original')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(BW4_opening_1, cmap='gray')
plt.title('Después de Opening')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(BW4_closing_2, cmap='gray')
plt.title('Después de Opening y Closing')
plt.axis('off')

plt.tight_layout()
plt.show()

## 7. Aplicaciones
### 7.1 Mejora de una huella digital ruidosa

En esta sección, aplicaremos operaciones morfológicas para mejorar una imagen de huella digital que contiene ruido. Utilizaremos operaciones de apertura y cierre para eliminar el ruido y mejorar la continuidad de las líneas en la huella.

In [ ]:
# Carga de la imagen binaria
BW5 = cv2.imread('./images/noisy_fingerprint.tif', cv2.IMREAD_GRAYSCALE)

# Asegurarse de que la imagen es binaria
_, BW5 = cv2.threshold(BW5, 127, 255, cv2.THRESH_BINARY_INV)

In [ ]:
# Creación del elemento estructurante en forma de cuadrado de tamaño 3x3
se_7 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

In [ ]:
# Aplicación de la operación de apertura para eliminar ruido
BW5_opening_1 = cv2.morphologyEx(BW5, cv2.MORPH_OPEN, se_7)

# Aplicación de la operación de cierre para cerrar pequeños huecos
BW5_closing_1 = cv2.morphologyEx(BW5_opening_1, cv2.MORPH_CLOSE, se_7)

In [ ]:
# Visualización de los resultados
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(BW5, cmap='gray')
plt.title('Imagen Original')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(BW5_opening_1, cmap='gray')
plt.title('Después de Opening')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(BW5_closing_1, cmap='gray')
plt.title('Después de Opening y Closing')
plt.axis('off')

plt.tight_layout()
plt.show()